In [1]:
#importations
import requests
import random
from random import randrange
import json
from langdetect import detect
import os
import glob
import gensim.downloader as api
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordPOSTagger
import re

In [2]:
def write_json(chemin,contenu):
    with open(chemin,"w",encoding="utf-8") as w :
        w.write(json.dumps(contenu, indent=2, ensure_ascii=False))
def ouvrir_json(chemin) :
    with open(chemin,encoding="utf-8") as f :
        toto = json.load(f)
    return toto

In [3]:
def requete_base(lang) : 
    nom = "requetes_chansons_"+lang+".json"
    if os.path.exists(nom) :
        return nom
    langues = {"fr":"Q150","en" :"Q1860" }
    #récupérer aussi l'auteur ?
    requete_all = """
    SELECT DISTINCT ?chanson ?chansonLabel ?performer ?performerLabel WHERE
    {
      ?chanson wdt:P7937 wd:Q7366;
               wdt:P175 ?performer;
               rdfs:label ?chansonLabel;
               wdt:P407 wd:%s.
       ?performer rdfs:label ?performerLabel.
      FILTER(lang(?chansonLabel)="%s").
      FILTER(lang(?performerLabel)="%s").
  }
    """%(langues[lang],lang,lang)

    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': requete_all, 'format': 'json'}).json()
    
    write_json(nom,data)
    return nom

In [4]:
def structuration_requete(fichier) :

    resultats_json = ouvrir_json(fichier)
    liste_niv1 = resultats_json["results"]["bindings"]
    
    resultats = {}
    chanteur = {}
   
    for un in liste_niv1 :
        if un['chanson']['value'] not in resultats :
            k = un['chanson']['value'].split("http://www.wikidata.org/entity/")[1]
            resultats[k] = (un['chansonLabel']['value'],un['performerLabel']['value'])
    """        
    for un in liste_niv1 :
        if un['performerLabel']['value'] not in chanteur :
            chanteur[un['performerLabel']['value']] = [un['chansonLabel']['value']]
        else :
            chanteur[un['performerLabel']['value']].append(un['chansonLabel']['value'])
   """     
    return resultats

In [5]:
#https://stackoverflow.com/questions/44468300/how-to-pos-tag-a-french-sentence
def etiquetage(titre):
    
    #l'apostrophe crée des bugs lors de la requête sparql
    titre = re.sub("'"," ",titre)
    
    #à mettre à jour selon la machine
    jar = 'C:/Users/phisi/stanford-tagger-4.2.0/stanford-postagger-full-2020-11-17/stanford-postagger-4.2.0.jar'
    #à mettre à jour selon la machine
    model = 'C:/Users/phisi/stanford-tagger-4.2.0/stanford-postagger-full-2020-11-17/models/french-ud.tagger'
    #à mettre à jour selon la machine
    java_path = "C:/Program Files/Java/jdk-18.0.1.1/bin/java.exe"
    
    os.environ['JAVAHOME'] = java_path
    pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8' )
    
    etiquetages = pos_tagger.tag(titre.split())
    
    return etiquetages


In [6]:
#fichier du glaff lourd (pas présent dans le répertoire ressources)
#il est disponible à l'adresse http://redac.univ-tlse2.fr/lexiques/glaff.html
def lemmatisation(fichier="ressources/glaff-1.2.2.txt") :

    f = open(fichier, encoding="utf-8")
    dico_lemmes = {}

    ligne = f.readline()
    while len(ligne)>1:

        #on recupère les différents éléments du GLAFF
        elems = re.split("\|", re.sub("\n", "", ligne))
        
        forme = elems[0]
        lemme = elems[2]
       
        #on considere que la premiere forme est la plus populaire (à vérifier)
        if forme not in dico_lemmes : 
            dico_lemmes[forme] = lemme
        ligne = f.readline()
    f.close()
    return dico_lemmes

In [20]:
def lexeme_jdm(lang,dico_reponses,dico_lemmes) :
    dico_langues = {'fr':'french','en':'english'}
    cle = random.choice(list(dico_reponses.keys()))
    reponse = dico_reponses[cle][0]
    
    print("Reponse : ",reponse)
   
    dico_jdm = {}
    rep_tagged = etiquetage(reponse)
    print("Etiquetage de la reponse :",rep_tagged)
    #print(rep_tagged)
    for r in rep_tagged : 
        if r[0] in stopwords.words(dico_langues[lang]) :
            dico_jdm[r[0]] = {r[0]}
        #on ne modifie pas les noms propres
        elif r[1] == 'PROPN' or r[1] == 'NUM':
            dico_jdm[r[0]] = {r[0]}
        else :
            #traitement de chaque token (lemme)
            mot = r[0].lower()
            if dico_lemmes.get(mot) != None :
                lemme = dico_lemmes[mot]
            else :
                lemme = mot
            requete_all = """
            PREFIX lexvo: <http://lexvo.org/id/iso639-3/>
            PREFIX dbnary: <http://kaiko.getalp.org/dbnary#>
            PREFIX lemon: <http://lemon-model.net/lemon#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

            select distinct ?lex ?syn ?ant{

                ?lexeme ontolex:canonicalForm ?f.
                ?f ontolex:writtenRep '%s'@%s.
                ?lexeme rdfs:label ?lex.

                OPTIONAL {
                    ?lexeme dbnary:synonym ?synonym.
                    ?synonym dbnary:describes ?synClass.
                    ?synClass rdfs:label ?syn.
                    FILTER(lang(?syn) = "%s")
                }

                OPTIONAL {
                    ?lexeme dbnary:antonym ?antonym.
                    ?antonym dbnary:describes ?antClass.
                    ?antClass rdfs:label ?ant.
                    FILTER(lang(?ant) = "%s")
                }

            }

            """%(lemme,lang,lang,lang)
            url = 'http://kaiko.getalp.org/sparql'
            data = requests.get(url, params={'query': requete_all, 'format': 'json'}).json()
            
            #si un mot n'a pas de synonymes / antonymes, on le note pour pouvoir ensuite exclure ce choix de reponse
            if len(data['results']['bindings']) == 0 :
                dico_jdm[r[0]] = {0}
            else :     
                i = 0
                for d in data['results']['bindings'] :
                    if i == 0 :
                        cle_token = d['lex']['value']
                        #print(cle_token)
                        dico_jdm[cle_token] = {}
                        dico_jdm[cle_token]['syn'] = set()
                        dico_jdm[cle_token]['ant'] = set()
                        i+=1
                    for k,v in d.items() :
                        if k != 'lex' and v['xml:lang'] == lang:
                            dico_jdm[cle_token][k].add(v['value'])
    print("-"*70)
    print("Résultat : ")                   
    return dico_jdm

In [18]:
dico_lemmes = lemmatisation()

In [26]:
dico_songs= structuration_requete('ressources/requetes_chansons_fr.json')
lexeme_jdm("fr",dico_songs,dico_lemmes)

Reponse :  À mourir pour mourir
Etiquetage de la reponse : [('À', 'ADP'), ('mourir', 'VERB'), ('pour', 'ADP'), ('mourir', 'VERB')]
----------------------------------------------------------------------
Résultat : 


{'à': {'syn': set(), 'ant': set()},
 'mourir': {'syn': {'avaler sa chique',
   'avaler son bulletin de naissance',
   'avaler son extrait de naissance',
   'avoir fait son temps',
   'calancher',
   'camarder',
   'canner',
   'casser sa pipe',
   'claboter',
   'clamser',
   'clapser',
   'claquer',
   'consumer',
   'crever',
   'dernière demeure',
   'disparaître',
   'décéder',
   'défuncter',
   'défunter',
   'expirer',
   'fermer les yeux',
   'finir',
   'lever les fourches',
   'monter au ciel',
   'partir',
   'partir aux fleurs',
   'passer de vie à trépas',
   'passer l’arme à gauche',
   'payer sa dette à la nature',
   'perdre la vie',
   'périr',
   'rendre le dernier soupir',
   'rendre l’âme',
   'rendre son âme à Dieu',
   'succomber',
   'trépasser',
   'éteindre',
   'être rappelé à Dieu'},
  'ant': {'démourir',
   'naître',
   'renaître',
   'ressusciter',
   'revivre',
   'se réincarner',
   'survivre',
   'vivre'}},
 'pour': {'pour'}}